<a href="https://colab.research.google.com/github/Manvikaul/corona-chatbot/blob/master/chatbot_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nltk

In [2]:
pip install newspaper3k

     |████████████████████████████████| 215kB 2.7MB/s 
     |████████████████████████████████| 7.4MB 97kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 194kB 45.1MB/s 
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=b29f7a2c1e7237c0f8cd0a868597a829bf5c30a6ea9a2eea733af1e6b473072e
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=153eecd4552150f128af778d4a58c1e57f5a8509bba18927b0cc11bac11ba8dd
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13539 sha256=19cf6b9494a26965a73d037a1619080fc1487fb5e50a4d3816f047ccb8c89dce
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c77

In [0]:
#IMPORTS
from newspaper import Article
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings

In [0]:
#Ignoring warnings
warnings.filterwarnings('ignore')

In [5]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
#article=Article('https://www.resmed.com/in/en/consumer/blogs/coronavirus-infection.html')
article=Article('https://www.medicalnewstoday.com/articles/256521')
article.download()
article.parse()
article.nlp()

corpus=article.text

print(corpus)

This article was updated on May 7, 2020 Coronaviruses typically affect the respiratory tracts of birds and mammals, including humans. Doctors associate them with the common cold, bronchitis, pneumonia, severe acute respiratory syndrome (SARS), and coronavirus disease 2019 (COVID-19). These viruses can also affect the gut. Coronaviruses usually cause the common cold, though they can be responsible for more severe illnesses. Over the past 70 years, scientists have found that these viruses can infect mice, rats, dogs, cats, turkeys, horses, pigs, and cattle. Sometimes, these animals transmit the viruses to humans. Most recently, authorities identified a new coronavirus outbreak in China that has now reached other countries. The virus is called severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) and it can cause COVID-19. In this article, we explain the different types of human coronavirus, their symptoms, and how people transmit them. We also focus on three particularly dangerous

In [7]:
text=corpus
sent_tokens=nltk.sent_tokenize(text)

print(sent_tokens)

['This article was updated on May 7, 2020 Coronaviruses typically affect the respiratory tracts of birds and mammals, including humans.', 'Doctors associate them with the common cold, bronchitis, pneumonia, severe acute respiratory syndrome (SARS), and coronavirus disease 2019 (COVID-19).', 'These viruses can also affect the gut.', 'Coronaviruses usually cause the common cold, though they can be responsible for more severe illnesses.', 'Over the past 70 years, scientists have found that these viruses can infect mice, rats, dogs, cats, turkeys, horses, pigs, and cattle.', 'Sometimes, these animals transmit the viruses to humans.', 'Most recently, authorities identified a new coronavirus outbreak in China that has now reached other countries.', 'The virus is called severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) and it can cause COVID-19.', 'In this article, we explain the different types of human coronavirus, their symptoms, and how people transmit them.', 'We also focus on

In [8]:
remove_punct_dict=dict((ord(punct),None) for punct in string.punctuation)

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [9]:
print(remove_punct_dict)

{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [10]:
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

print(LemNormalize(text))

['this', 'article', 'was', 'updated', 'on', 'may', '7', '2020', 'coronaviruses', 'typically', 'affect', 'the', 'respiratory', 'tracts', 'of', 'birds', 'and', 'mammals', 'including', 'humans', 'doctors', 'associate', 'them', 'with', 'the', 'common', 'cold', 'bronchitis', 'pneumonia', 'severe', 'acute', 'respiratory', 'syndrome', 'sars', 'and', 'coronavirus', 'disease', '2019', 'covid19', 'these', 'viruses', 'can', 'also', 'affect', 'the', 'gut', 'coronaviruses', 'usually', 'cause', 'the', 'common', 'cold', 'though', 'they', 'can', 'be', 'responsible', 'for', 'more', 'severe', 'illnesses', 'over', 'the', 'past', '70', 'years', 'scientists', 'have', 'found', 'that', 'these', 'viruses', 'can', 'infect', 'mice', 'rats', 'dogs', 'cats', 'turkeys', 'horses', 'pigs', 'and', 'cattle', 'sometimes', 'these', 'animals', 'transmit', 'the', 'viruses', 'to', 'humans', 'most', 'recently', 'authorities', 'identified', 'a', 'new', 'coronavirus', 'outbreak', 'in', 'china', 'that', 'has', 'now', 'reached'

In [0]:
GREETING_INPUTS=['hey','hi','hello','heya','howdy','whatsup','wassup','hola']
GREETING_OUTPUTS=['hey!','hello!','heya!','howdy!','hey there!','holaa!']


In [0]:
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in GREETING_INPUTS:
      return random.choice(GREETING_OUTPUTS)


In [0]:
def response(user_response):
  user_response.lower()
  #print(user_response)

  robo_response=''
  sent_tokens.append(user_response)
  #print(sent_tokens)

  tfidfvec=TfidfVectorizer(tokenizer=LemNormalize,stop_words='english')
  tfidf=tfidfvec.fit_transform(sent_tokens)
  #print(tfidf)

  #Get similarity scores (user's response with all other tokens)
  vals=cosine_similarity(tfidf[-1],tfidf)
  #print(vals)

  idx=vals.argsort()[0][-2]
  #we give -2 as argument since -1 will give the sentence with max similarity, and that would be the sentence itself(since it is appended at the end)
  #hence we use -2 which give the second most similar sentence

  #reduce the dimensionality of vals
  flat=vals.flatten()

  #sort the list in ascending order
  flat.sort()

  #Get the most similar score to user response
  score=flat[-2]
  #print(score)

  #if score==0 => no text similar to user response
  if(score==0):
    robo_response=robo_response+"I am sorry, I don't quite understand"

  else:
    robo_response=robo_response+sent_tokens[idx]

  #print(robo_response)

  sent_tokens.remove(user_response)

  return robo_response



In [14]:
flag=True

print("Bot: Hey! I'm M.A.R.T.I.N. I'm here to clear your doubts on corona virus. If you wish to exit, type 'bye'.")

while(flag==True):
  user_response=input()
  user_response=user_response.lower()

  if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you'):
      print("Bot: You're welcome!")

    else:
      if(greeting(user_response)!=None):
        print("Bot: "+greeting(user_response ))
      else:
        print("Bot: "+response(user_response))
        
  else:
    flag=False
    print("Bot: Talk to ya later!")



Bot: Hey! I'm M.A.R.T.I.N. I'm here to clear your doubts on corona virus. If you wish to exit, type 'bye'.
hi
Bot: holaa!
hey
Bot: hey!
how to prevent corona virus
Bot: Covering the mouth and nose with a tissue or handkerchief while coughing or sneezing can also help prevent transmission.
how does corona virus spread
Bot: This will help slow the spread of the virus from asymptomatic people and people who do not know that they have contracted it.
how to cure corona virus
Bot: There is currently no cure for the cold-like illnesses caused by coronaviruses.
how are you?
Bot: I am sorry, I don't quite understand
thank you
Bot: You're welcome!
what is corona virus
Bot: However, scientists have now replicated the virus.
what is covid-19
Bot: COVID-19 In 2019, the Centers for Disease Control and Prevention (CDC) started monitoring the outbreak of a new coronavirus, SARS-CoV-2, which causes COVID-19.
what are the symptoms of corona virus
Bot: Symptoms of COVID-19 According to the CDC, people ma